In [1]:
!pip install -q google-colab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.4 MB/s eta 0:00:00


In [2]:
import numpy as np
from IPython.display import display, HTML, Markdown, clear_output
import ipywidgets as widgets

try:
    from google.colab import ai
except Exception:
    ai = None
    print("Warning: google.colab.ai is not available in this runtime. The AI widget will not work here.")


In [3]:
np.random.seed(0)
real = np.random.normal(0, 1, (100, 64))
fake_A = np.random.normal(0.2, 1, (100, 64))
fake_B = np.random.normal(-0.2, 1, (100, 64))

In [4]:
def compute_fid(real, fake):
    """
    Simplified FID-like metric:
    - uses squared difference of means
    - uses elementwise sqrt for covariance term to remain beginner-friendly
    Note: This is not the exact FID algorithm (matrix sqrt omitted).
    """
    mu_r, mu_f = real.mean(axis=0), fake.mean(axis=0)
    cov_r, cov_f = np.cov(real, rowvar=False), np.cov(fake, rowvar=False)

    cov_term = np.trace(cov_r + cov_f - 2 * np.sqrt(np.abs(cov_r * cov_f)))
    return np.sum((mu_r - mu_f)**2) + cov_term

In [5]:
def compute_is(fake):
    """
    Toy Inception-Score-style metric:
    - normalize absolute values per sample to get pseudo-probabilities
    - compute KL between p(y|x) and marginal p(y)
    - return exp(mean KL)
    """
    p_yx = np.abs(fake) / np.sum(np.abs(fake), axis=1, keepdims=True)
    p_y = p_yx.mean(axis=0)

    kl = np.sum(p_yx * (np.log(p_yx + 1e-8) - np.log(p_y + 1e-8)), axis=1)
    return np.exp(kl.mean())

In [6]:
def compute_prd(real, fake):
    """
    Very simple precision/recall proxies:
    - precision: average distance of fake examples to the real mean (lower is better)
    - recall: average distance of real examples to the fake mean
    - convert to 0..1 by inversion to make "higher is better"
    """
    pr = np.mean(np.linalg.norm(fake - real.mean(axis=0), axis=1))
    rc = np.mean(np.linalg.norm(real - fake.mean(axis=0), axis=1))

    precision = 1 / (1 + pr)
    recall = 1 / (1 + rc)

    return precision, recall

In [7]:
fid_A = compute_fid(real, fake_A)
is_A = compute_is(fake_A)
p_A, r_A = compute_prd(real, fake_A)

fid_B = compute_fid(real, fake_B)
is_B = compute_is(fake_B)
p_B, r_B = compute_prd(real, fake_B)

print("=============== MODEL COMPARISON ===============")
print(f"Model A → FID: {fid_A:.4f},  IS: {is_A:.4f},  Precision: {p_A:.4f}, Recall: {r_A:.4f}")
print(f"Model B → FID: {fid_B:.4f},  IS: {is_B:.4f},  Precision: {p_B:.4f}, Recall: {r_B:.4f}")

print("\n=============== BEST MODEL ===============")
print("BEST FID (lower is better):    ", "A" if fid_A < fid_B else "B")
print("BEST IS (higher is better):    ", "A" if is_A > is_B else "B")
print("BEST PRECISION (higher better):", "A" if p_A > p_B else "B")
print("BEST RECALL (higher better):   ", "A" if r_A > r_B else "B")


=============== MODEL COMPARISON ===============
Model A → FID: 4.1066,  IS: 1.3345,  Precision: 0.1097, Recall: 0.1104
Model B → FID: 3.7315,  IS: 1.3209,  Precision: 0.1110, Recall: 0.1108

=============== BEST MODEL ===============
BEST FID (lower is better):     B
BEST IS (higher is better):     A
BEST PRECISION (higher better): B
BEST RECALL (higher better):    B


In [8]:
if ai is None:
    print("google.colab.ai is not available in this environment — this widget only works in Colab with AI features enabled.")
else:
    dropdown = widgets.Dropdown(
        options=[],
        layout={'width': 'auto'}
    )

    def update_model_list(new_options):
        dropdown.options = new_options

    try:
        update_model_list(ai.list_models())
    except Exception as e:
        update_model_list([])
        print("Could not list models:", type(e).__name__, e)

    text_input = widgets.Textarea(
        placeholder='Ask me anything....',
        layout={'width': 'auto', 'height': '100px'},
    )

    button = widgets.Button(
        description='Submit Text',
        disabled=False,
        tooltip='Click to submit the text',
        icon='check'
    )

    output_area = widgets.Output(
         layout={'width': 'auto', 'max_height': '300px','overflow_y': 'scroll'}
    )

    def on_button_clicked(b):
        with output_area:
            output_area.clear_output(wait=False)
            accumulated_content = ""
            try:
                for new_chunk in ai.generate_text(prompt=text_input.value, model_name=dropdown.value, stream=True):
                    if new_chunk is None:
                        continue
                    accumulated_content += new_chunk
                    clear_output(wait=True)
                    display(Markdown(accumulated_content))
            except Exception as e:
                print("Error during generation:", type(e).__name__, e)

    button.on_click(on_button_clicked)
    vbox = widgets.GridBox([dropdown, text_input, button, output_area])

    display(HTML("""
    <style>
    .widget-dropdown select {
        font-size: 18px;
        font-family: "Arial", sans-serif;
    }
    .widget-textarea textarea {
        font-size: 18px;
        font-family: "Arial", sans-serif;
    }
    </style>
    """))
    display(vbox)

GridBox(children=(Dropdown(layout=Layout(width='auto'), options=('google/gemini-2.5-flash', 'google/gemini-2.5…